In [1]:
import torch
from torch.autograd import Variable
import torch.nn.functional as F
import torch.utils.data as Data
from torchvision import datasets, transforms, models
import matplotlib.pyplot as plt
%matplotlib qt
import numpy as np
import imageio
import pandas as pd
# from data_prep_gasflow import features, targets, features_test, targets_test
from torchsummary import summary

from scipy import optimize
import matlab.engine
eng = matlab.engine.start_matlab()
import numpy as np
from scipy.optimize import minimize
import numpy.matlib 
import timeit, time

from scipy.optimize import Bounds
from scipy.optimize import BFGS
from scipy.optimize import NonlinearConstraint

## Import trained deep neural net

In [2]:
# path = 'C:/Users/david.wagner/Desktop/Biomasse-Institut/Machine_Learning_Corr/checkpoint_ohneneu_16dim.pth.tar'
path = 'checkp_xy_mit_16dim.pth.tar'
path1 = 'checkpoint_xy_16dim_101920.pth.tar' # relative path

In [28]:
# map_location=torch.device('cpu')
checkpoint = torch.load(path,  map_location=lambda storage, loc: storage)
model_structure = print(checkpoint['model'])

# checkpoint = torch.load(path1,  map_location=lambda storage, loc: storage)
# model_structure = print(checkpoint['model'])

Sequential(
  (0): Linear(in_features=4, out_features=512, bias=True)
  (1): ReLU()
  (2): Linear(in_features=512, out_features=512, bias=True)
  (3): ReLU()
  (4): Linear(in_features=512, out_features=256, bias=True)
  (5): ReLU()
  (6): Linear(in_features=256, out_features=256, bias=True)
  (7): ReLU()
  (8): Linear(in_features=256, out_features=128, bias=True)
  (9): ReLU()
  (10): Linear(in_features=128, out_features=64, bias=True)
  (11): ReLU()
  (12): Linear(in_features=64, out_features=64, bias=True)
  (13): ReLU()
  (14): Linear(in_features=64, out_features=32, bias=True)
  (15): ReLU()
  (16): Linear(in_features=32, out_features=16, bias=True)
  (17): ReLU()
  (18): Linear(in_features=16, out_features=16, bias=True)
)


D:\Anaconda\lib\site-packages\torch\serialization.py:657: SourceChangeWarning: source code of class 'torch.nn.modules.linear.Linear' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
D:\Anaconda\lib\site-packages\torch\serialization.py:657: SourceChangeWarning: source code of class 'torch.nn.modules.activation.ReLU' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)


In [29]:
# another way to define a network
net = torch.nn.Sequential(
        torch.nn.Linear(4, 512),
        torch.nn.ReLU(),
        torch.nn.Linear(512, 512),
        torch.nn.ReLU(),
        torch.nn.Linear(512, 256),
        torch.nn.ReLU(),
        torch.nn.Linear(256, 256),
        torch.nn.ReLU(),
        torch.nn.Linear(256, 128),
        torch.nn.ReLU(),
        torch.nn.Linear(128, 64),
        torch.nn.ReLU(),
        torch.nn.Linear(64, 64),
        torch.nn.ReLU(),
        torch.nn.Linear(64, 32),
        torch.nn.ReLU(),
        torch.nn.Linear(32, 16),
        torch.nn.ReLU(),
        torch.nn.Linear(16, 16),
    )

In [30]:
model = net
model.load_state_dict(checkpoint['state_dict'])
model.eval()
summary(model, (2,4)) # 2 Dimensionen und 4 Inputs

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1               [-1, 2, 512]           2,560
              ReLU-2               [-1, 2, 512]               0
            Linear-3               [-1, 2, 512]         262,656
              ReLU-4               [-1, 2, 512]               0
            Linear-5               [-1, 2, 256]         131,328
              ReLU-6               [-1, 2, 256]               0
            Linear-7               [-1, 2, 256]          65,792
              ReLU-8               [-1, 2, 256]               0
            Linear-9               [-1, 2, 128]          32,896
             ReLU-10               [-1, 2, 128]               0
           Linear-11                [-1, 2, 64]           8,256
             ReLU-12                [-1, 2, 64]               0
           Linear-13                [-1, 2, 64]           4,160
             ReLU-14                [-1

## Import and normalize test-data

In [31]:
data_raw= pd.read_csv('C:/Users/david.wagner/Desktop/Biomasse-Institut/Machine_Learning_Corr/gasflowdata_mitA_16dim.csv')
# data_raw= pd.read_csv('gasflowdata_mitA_16dim.csv')

In [32]:
gas = data_raw
for ii in range(16):
    gas = gas.drop(['time_'+str(ii+1)], axis=1)
#     print(ii)
    
# gas.head()

In [33]:
torch.manual_seed(0)  

In [34]:
# normalisieren

max_field_dil = []

for field in ['dilution']:
# for field in ['dilution', 'Gasflow_end']:
    max_field_dil = gas[field].max()
    gas.loc[:,field] = (gas[field])/max_field_dil

In [35]:
# gas.head()

In [36]:
gasflow_list = []
for ii in range(16):
    gasflow_list.append('Gasflow_' + str(ii+1))

# print(gasflow_list)

In [37]:
for field in [gasflow_list]:
# for field in ['dilution', 'Gasflow_end']:
    max_field = gas[field].max()
    gas.loc[:,field] = (gas[field])/max_field

# gas.head()

In [38]:
# print(max_field, max_field_dil)

In [39]:
sample = np.random.choice(gas.index, size=int(len(gas)*0.9), replace=False)
data_raw_1, test_data_1 = gas.loc[sample], gas.drop(sample)

# # split into features and targets
columns = gasflow_list

features, targets = data_raw_1.drop(columns, axis=1), data_raw_1[columns]
features_test, targets_test = test_data_1.drop(columns, axis=1), test_data_1[columns]

In [40]:
# features_test.head()

In [41]:
# targets_test.head()

In [42]:
# x_data = torch.tensor(features.values, dtype=torch.float)
# y_data = torch.tensor(targets.values, dtype=torch.float)

# x_test = torch.tensor(features_test.values, dtype=torch.float)
# y_test = torch.tensor(targets_test.values, dtype=torch.float)

x_data = checkpoint['x_data']
y_data = checkpoint['y_data']
x_test = checkpoint['x_test']
y_test = checkpoint['y_test']

## Test loaded net

In [43]:
def my_loss(output, target):
    loss = ((output - target)**2).sum()
    return loss

In [44]:
x_test[0]

tensor([0.8147, 0.0791, 0.1062, 0.1338])

In [45]:
y_test[0]

tensor([6.4523e-01, 6.8614e-05, 9.6559e-06, 7.8648e-06, 7.5854e-06, 7.5538e-06,
        7.5499e-06, 7.5493e-06, 7.5492e-06, 7.5492e-06, 7.5492e-06, 7.5492e-06,
        7.5492e-06, 7.5492e-06, 7.5492e-06, 7.5492e-06])

In [46]:
prediction_test = net(x_test)
print(prediction_test[0])
print(y_test[0])

tensor([ 6.4617e-01, -4.7266e-05,  1.5363e-05,  2.0117e-05,  2.0280e-05,
         1.9491e-05,  1.8433e-05,  1.8865e-05,  1.8224e-05,  1.7911e-05,
         1.6913e-05,  1.9178e-05,  1.8597e-05,  1.8895e-05,  1.8209e-05,
         1.7241e-05], grad_fn=<SelectBackward>)
tensor([6.4523e-01, 6.8614e-05, 9.6559e-06, 7.8648e-06, 7.5854e-06, 7.5538e-06,
        7.5499e-06, 7.5493e-06, 7.5492e-06, 7.5492e-06, 7.5492e-06, 7.5492e-06,
        7.5492e-06, 7.5492e-06, 7.5492e-06, 7.5492e-06])


In [47]:
my_loss(prediction_test[0], y_test[0])

tensor(8.9905e-07, grad_fn=<SumBackward0>)

In [48]:
# features_test.head()

In [49]:
# targets_test.head()

In [50]:
features_test_tensor = torch.tensor(features_test.values, dtype=torch.float)
# print(features_test_tensor)

In [53]:
# pred_sample = net(features_test_tensor)
pred = net(x_test)
# pred_sample = pred_sample.detach().numpy()
# print(pred_sample)
# targets_test_tensor = torch.tensor(targets_test.values, dtype=torch.float)
# targets_test_tensor = torch.tensor(targets_test.values, dtype=torch.float)
# print(targets_test_tensor[0])

In [54]:
my_loss(pred, y_test)

tensor(85.5266, grad_fn=<SumBackward0>)

In [55]:
# print(x_test)

In [56]:
# features_np = np.array(features_test)

## Find optimal dilution at specific substrate values

In [57]:
# model_ADM = 'C:\Users\david.wagner\Desktop\Biomasse-Institut\Machine_Learning_Corr\Netze_Regressionen_ASIM\ADM_model\ADM_Sim.m'

In [58]:
# carbo = features_np[0, 0]
# protein = features_np[0, 1]
# inert_x = 0.0
# inert_s = 0.0
# fat = features_np[0, 2]

# dilution = features_np[0, 3] * max_field_dil
# sub = [carbo, protein, fat, dilution]
# time_end = 60.0

carbo = 0.8147
protein = 0.0791
inert_x = 0.0
inert_s = 0.0
fat = 0.1062

dilution = 0.1338
sub = [carbo, protein, fat, dilution]
time_end = 60.0

In [59]:
# print(sub)

In [60]:
t_full = np.linspace(0,300,100)
t_red_1 = t_full[1:49:5]
t_red_2 = t_full[49:100:10]
# print(t_red_1, t_red_2)

t_red = np.concatenate((t_red_1, t_red_2), axis=0)
# print(t_red)

In [61]:
gg_test = eng.ADM_Sim_ML_remote(carbo, protein, fat, dilution)

In [62]:
gg_np = np.array(gg_test)
gg_np_red_1 = np.array(gg_test[1:49:5])
gg_np_red_2 = np.array(gg_test[49:len(t_full):10])

gg_np_red = np.concatenate((gg_np_red_1 ,gg_np_red_2), axis=0)

In [63]:
# print(gg_np, gg_np_red)

In [64]:
gg_np.shape

(100, 2)

In [65]:
# print(max_field)

In [66]:
norm_gg = gg_np_red[:,0]
gg_norm = []
dil_norm = []

gg_norm.append(norm_gg[:]/max_field.values) 
# print(gg_norm)

In [67]:
fig, ax = plt.subplots()
ax.plot(gg_np_red[:,1], np.reshape(norm_gg, (16,1)), 'b.')
ax.plot(gg_np[:,1], np.reshape(gg_np[:,0], (100,1)), 'r-.')
ax.set_xlabel('Time [d]')
ax.set_ylabel('Concentration [m³/d]')
plt.show()

In [68]:
setpoint = 1500 # m³/d
set_ch4_DNN = setpoint/max_field[-1]

In [69]:
print(set_ch4_DNN)

0.5835784854217567


In [70]:
print(sub)

[0.8147, 0.0791, 0.1062, 0.1338]


In [71]:
DNN_test = torch.tensor(sub, dtype=torch.float)

In [72]:
max_f = max_field[-1]
target_ch4 = setpoint
sub = [0.8147, 0.0791, 0.1062, 0]

print(max_f, target_ch4)

2570.34835497052 1500


In [73]:
def ch4_ADM_DNN(Dil_test, max_f, target_ch4, sub):
    """ADM DNN Difference"""
#     print(Dil_test)
    dil = float(Dil_test)
    sub[3] = dil
    DNN_test = torch.tensor(sub, dtype=torch.float)
    
    pred_sample_opt = net(DNN_test)
    pred_sample_opt = pred_sample_opt.detach().numpy()
    return (target_ch4 - (pred_sample_opt[-1] * max_f))**2


result_list = []
target_list = []

bounds = Bounds([0], [0.2])

start_proc = []
ende_proc = []

start_time = time.time()
sub = [0.8147, 0.0791, 0.1062, 0]

for iv in range(20):
    x0 = np.array([iv/100])
    res = minimize(ch4_ADM_DNN, x0, args=(max_f, target_ch4, sub), method='BFGS', options={'disp': True})
    result_list.append([float(res.x), res.fun])

end_time = time.time()
print('Systemzeit: {:5.3f}s'.format(end_time - start_time))
print(result_list)

         Current function value: 0.000000
         Iterations: 3
         Function evaluations: 62
         Gradient evaluations: 25
         Current function value: 0.000000
         Iterations: 6
         Function evaluations: 60
         Gradient evaluations: 24
         Current function value: 0.000000
         Iterations: 8
         Function evaluations: 99
         Gradient evaluations: 43
         Current function value: 0.000000
         Iterations: 4
         Function evaluations: 62
         Gradient evaluations: 25
Optimization terminated successfully.
         Current function value: 244676.268411
         Iterations: 1
         Function evaluations: 6
         Gradient evaluations: 3
Optimization terminated successfully.
         Current function value: 3626.769905
         Iterations: 1
         Function evaluations: 6
         Gradient evaluations: 3
Optimization terminated successfully.
         Current function value: 820703.594111
         Iterations: 1
         Funct

In [74]:
results_min = np.array(result_list)
print(results_min)

np.min(results_min)
result = numpy.where(results_min == numpy.amin(results_min[:,1]))
print(result)

listOfCordinates = list(zip(result[0], result[1]))
print(listOfCordinates[0][0])

dil_fun = results_min[listOfCordinates[0][0]]
print(dil_fun)

[[ 2.76332747e-02  1.24475651e-07]
 [ 2.76332740e-02  1.24475651e-07]
 [ 2.76332808e-02  2.15311488e-09]
 [ 2.76332750e-02  1.24475651e-07]
 [-2.89438303e-01  2.44676268e+05]
 [-4.94440654e-01  3.62676990e+03]
 [-9.50000000e-01  8.20703594e+05]
 [ 7.00000000e-02  2.24861551e+06]
 [ 8.00000000e-02  2.24935647e+06]
 [ 9.01799547e-02  2.24939025e+06]
 [ 1.00000000e-01  2.24993807e+06]
 [ 1.44814844e-01  2.24989808e+06]
 [ 1.20000000e-01  2.24992221e+06]
 [ 1.30000000e-01  2.24987843e+06]
 [ 1.40000000e-01  2.24984523e+06]
 [ 1.90993737e-01  2.24991888e+06]
 [ 1.60000000e-01  2.24986005e+06]
 [-5.32088402e-01  3.80572761e+02]
 [ 1.89431460e-01  2.24980524e+06]
 [ 1.90000000e-01  2.24984661e+06]]
(array([2], dtype=int64), array([1], dtype=int64))
2
[2.76332808e-02 2.15311488e-09]


In [75]:
gg_opt_dil_endpoint = eng.ADM_Sim_ML_remote(float(sub[0]), float(sub[1]), float(sub[2]), float(dil_fun[0]))
gg_ADM_np_endpoint = np.array(gg_opt_dil_endpoint)
print(gg_ADM_np_endpoint)

[[1705.35487687    0.        ]
 [ 523.27119622    3.03030303]
 [1213.38521457    6.06060606]
 [2225.75531715    9.09090909]
 [1753.31161673   12.12121212]
 [1470.87173611   15.15151515]
 [1476.58041502   18.18181818]
 [1480.72421348   21.21212121]
 [1483.64551649   24.24242424]
 [1485.96974642   27.27272727]
 [1487.93751985   30.3030303 ]
 [1489.64657536   33.33333333]
 [1491.14441738   36.36363636]
 [1492.46048985   39.39393939]
 [1493.61708946   42.42424242]
 [1494.63300943   45.45454545]
 [1495.52476831   48.48484848]
 [1496.30705911   51.51515152]
 [1496.99296304   54.54545455]
 [1497.59410131   57.57575758]
 [1498.12076904   60.60606061]
 [1498.58206456   63.63636364]
 [1498.9860115    66.66666667]
 [1499.33967461   69.6969697 ]
 [1499.64926669   72.72727273]
 [1499.92024523   75.75757576]
 [1500.15740215   78.78787879]
 [1500.36493986   81.81818182]
 [1500.54654392   84.84848485]
 [1500.70544503   87.87878788]
 [1500.84447436   90.90909091]
 [1500.96611027   93.93939394]
 [1501.0

In [76]:
sub[3] = dil_fun[0]

DNN_endpoint = torch.tensor(sub, dtype=torch.float)
pred_sample_endpoint = net(DNN_endpoint)
pred_sample_endpoint = pred_sample_endpoint.detach().numpy()
pred_sample_endpoint = (pred_sample_endpoint * max_f)

print(pred_sample_endpoint)

[ 515.7944 1338.411  1344.5698 1495.1083 1497.1091 1498.3658 1499.1113
 1499.4316 1499.6815 1499.823  1499.9464 1499.791  1499.8733 1499.8513
 1499.9117 1500.    ]


In [77]:
# Definition der Referenztrajektorie
def ch4_ADM(Dil, max_f, target_ch4, sub):
    """ADM DNN Difference"""
#     print(Dil_test)
    dil = float(Dil)
    sub[3] = dil
    gg_opt_test = eng.ADM_Sim_ML_remote(float(sub[0]), float(sub[1]), float(sub[2]), float(Dil))
    return (target_ch4 - np.array(gg_opt_test[-1][0]))**2


result_list_ADM = []
target_list_ADM = []
bounds = Bounds([0], [0.2])
start_proc_1 = []
ende_proc_1 = []

start_time_1  = time.time()

x1_1 = np.array([0.02])
res1 = minimize(ch4_ADM, x1_1, args=(max_f, target_ch4, sub), method='BFGS', options={'disp': True})
result_list_ADM.append([float(res1.x), res1.fun])

# for iv in range(5):
#     x1_1 = np.array([iv/100])
#     res1 = minimize(ch4_ADM, x1_1, args=(max_f, target_ch4, sub), method='BFGS', options={'disp': True})
#     result_list_ADM.append([float(res1.x), res1.fun])

end_time_1  = time.time()
print('Systemzeit: {:5.3f}s'.format(end_time_1 - start_time_1))
print(result_list_ADM)

         Current function value: 0.000000
         Iterations: 4
         Function evaluations: 109
         Gradient evaluations: 49
Systemzeit: 193.465s
[[0.02759341084204177, 7.434404147794414e-09]]


In [78]:
results_min_ADM = np.array(result_list_ADM)
print(results_min_ADM)

np.min(results_min_ADM)
result = numpy.where(results_min_ADM == numpy.amin(results_min_ADM[:,1]))
print(result)

listOfCordinates = list(zip(result[0], result[1]))
print(listOfCordinates[0][0])

dil_fun_ADM = results_min_ADM[listOfCordinates[0][0]]
print(dil_fun_ADM)

[[2.75934108e-02 7.43440415e-09]]
(array([0], dtype=int64), array([1], dtype=int64))
0
[2.75934108e-02 7.43440415e-09]


In [79]:
gg_opt_dil = eng.ADM_Sim_ML_remote(float(sub[0]), float(sub[1]), float(sub[2]), float(dil_fun_ADM[0]))

In [80]:
gg_ADM_np = np.array(gg_opt_dil)
set_repd = np.matlib.repmat(np.array(setpoint), gg_ADM_np.shape[0], 1)

In [81]:
# print(set_repd)

In [82]:
gg_opt_dil_discrete = eng.ADM_Sim_ML_remote_discrete(float(sub[0]), float(sub[1]), float(sub[2]), float(dil_fun_ADM[0]), float(t_red[0]))

In [83]:
# print(gg_opt_dil_discrete)

In [84]:
t_neu = np.linspace(0,t_red[0],100)

In [85]:
fig, ax = plt.subplots()
ax.plot(t_neu, np.reshape(np.array(gg_opt_dil_discrete)[:,0], (100,1)), 'b-.')
ax.set_xlabel('Time [d]')
ax.set_ylabel('Concentration [m³/d]')
plt.show()

In [86]:
fig, ax = plt.subplots()
ax.plot(gg_ADM_np[:,1], np.reshape(gg_ADM_np[:,0], (100,1)), 'b-.')
ax.plot(gg_ADM_np[:,1], np.reshape(set_repd, (100,1)), 'r-')
ax.set_xlabel('Time [d]')
ax.set_ylabel('Concentration [m³/d]')
plt.show()

In [87]:
sub = [0.8147, 0.0791, 0.1062, 0.02759341084204177]
DNN_test = torch.tensor(sub, dtype=torch.float)
pred_sample_opt = net(DNN_test)
pred_sample_opt = pred_sample_opt.detach().numpy()

In [88]:
print(pred_sample_opt * max_f)

[ 515.30096 1336.721   1342.8506  1493.1967  1495.194   1496.4518
 1497.1993  1497.5186  1497.7699  1497.9133  1498.0381  1497.8778
 1497.962   1497.9393  1498.0006  1498.0907 ]


In [89]:
def ch4_ADM_DNN_mpc(Dil_test, max_f, target_ch4, sub, i):
    """ADM DNN Difference"""
    dil = float(Dil_test)
    sub[3] = dil
    DNN_test = torch.tensor(sub, dtype=torch.float)
    
    pred_sample_opt = net(DNN_test)
    pred_sample_opt = pred_sample_opt.detach().numpy()
    return (target_ch4 - (pred_sample_opt[i] * max_f))**2

# ende_proc = time.process_time()
# end_time = time.time()
# print('Systemzeit: {:5.3f}s'.format(end_time - start_time))
# print(result_list)

In [90]:
i = 0
di = 1

result_mpc = []

gg_opt_red_1 = np.array(gg_ADM_np[1:49:5])
gg_opt_red_2 = np.array(gg_ADM_np[49:100:10])
gg_opt_red = np.concatenate((gg_opt_red_1 ,gg_opt_red_2), axis=0)

print(gg_opt_red)

result_ADM = []
result_DNN = []

while i < len(t_red):
    
    result_list = []
    results_min = []
    result = []
    listOfCordinates = []
    dil_fun = []
    
#     for iv in range(20):
#         x0 = np.array([iv/100])
#         res = minimize(ch4_ADM_DNN_mpc, x0, args=(max_f, gg_opt_red[i][0], sub, i), method='BFGS', options={'disp': True})
#         result_list.append([float(res.x), res.fun])
        
    bounds_1 = [(0, 0.2)]
    results_2 = dict()
#         start_proc = time.time()
    results_2['shgo_sobol'] = optimize.shgo(ch4_ADM_DNN_mpc, bounds_1, args=(max_f, gg_opt_red[i][0], sub, i), n=20, iters=60,
                                          sampling_method='sobol')
    result_list.append([float(results_2['shgo_sobol'].x), results_2['shgo_sobol'].fun])
    
    sub[3] = results_2['shgo_sobol'].x
    DNN_test = torch.tensor(sub, dtype=torch.float)
    pred_sample_opt = net(DNN_test)
    pred_sample_opt = pred_sample_opt.detach().numpy()
    result_DNN.append([(pred_sample_opt[i] * max_f).tolist(), t_red[i]])
        
#     results_min = np.array(result_list)
#     result = numpy.where(results_min == numpy.amin(results_min[:,1]))
#     listOfCordinates = list(zip(result[0], result[1]))
#     dil_fun = results_min[listOfCordinates[0][0]]
    
#     print(dil_fun)
#     break
    dil_fun = float(results_2['shgo_sobol'].x)
    print(dil_fun)  
    
#     gg_opt_dil_discrete_red = eng.ADM_Sim_ML_remote_discrete(float(sub[0]), 
#                                                          float(sub[1]), float(sub[2]), 
#                                                          float(dil_fun[0]), float(t_red[i]))
    gg_opt_dil_discrete_red = eng.ADM_Sim_ML_remote_discrete(float(sub[0]), 
                                                         float(sub[1]), float(sub[2]), 
                                                         float(dil_fun), float(t_red[i]))
    print(gg_opt_dil_discrete_red[-1])
#     result_mpc.append([dil_fun.tolist()])
    result_mpc.append([dil_fun, results_2['shgo_sobol'].fun])
    result_ADM.append([gg_opt_dil_discrete_red[-1]])
    i += di
    

[[ 522.77367063    3.03030303]
 [1474.77411586   18.18181818]
 [1487.82888827   33.33333333]
 [1493.7044664    48.48484848]
 [1496.76218353   63.63636364]
 [1498.3386721    78.78787879]
 [1499.14845409   93.93939394]
 [1499.56376099  109.09090909]
 [1499.77659228  124.24242424]
 [1499.88561369  139.39393939]
 [1499.92345761  148.48484848]
 [1499.97994915  178.78787879]
 [1499.99475344  209.09090909]
 [1499.99863179  239.39393939]
 [1499.99964762  269.6969697 ]
 [1499.99991378  300.        ]]
0.02819874247480169
[530.3489517020652,3.0303030303030303]
0.030859375
[1620.8726528895288,18.18181818181818]
0.031066059149329306
[1643.8472611086206,33.333333333333336]
0.02763671875
[1495.681702587879,48.484848484848484]
0.02762605281851725
[1498.2521916627265,63.63636363636363]
0.027632713921982934
[1500.1315496535046,78.78787878787878]
0.027634053893953636
[1501.0013505577974,93.93939393939394]
0.027636031193653775
[1501.5059297405235,109.0909090909091]
0.027635263614949255
[1501.683226850817,

In [91]:
result_mpc_np = np.array(result_mpc).reshape(i,2)
print(result_mpc_np)

result_ADM_np = np.array(result_ADM).reshape(i,2)
print(result_ADM_np)

result_DNN_np = np.array(result_DNN).reshape(i,2)
print(result_DNN_np)

[[2.81987425e-02 1.35103503e-10]
 [3.08593750e-02 1.26165621e+01]
 [3.10660591e-02 7.95883908e-08]
 [2.76367188e-02 2.46070224e+00]
 [2.76260528e-02 4.46411372e-08]
 [2.76327139e-02 4.93798435e-09]
 [2.76340539e-02 1.49206496e-10]
 [2.76360312e-02 8.27050322e-08]
 [2.76352636e-02 2.70082603e-08]
 [2.76345826e-02 6.61532728e-08]
 [2.76328043e-02 1.13135413e-08]
 [2.76372209e-02 4.27335300e-09]
 [2.76358111e-02 9.93608080e-08]
 [2.76363449e-02 1.32092326e-07]
 [2.76351078e-02 9.41524750e-08]
 [2.76332759e-02 7.10692040e-08]]
[[ 530.3489517     3.03030303]
 [1620.87265289   18.18181818]
 [1643.84726111   33.33333333]
 [1495.68170259   48.48484848]
 [1498.25219166   63.63636364]
 [1500.13154965   78.78787879]
 [1501.00135056   93.93939394]
 [1501.50592974  109.09090909]
 [1501.68322685  124.24242424]
 [1501.7608791   139.39393939]
 [1501.71760742  148.48484848]
 [1501.97493826  178.78787879]
 [1501.92547337  209.09090909]
 [1501.9536261   239.39393939]
 [1501.8983126   269.6969697 ]
 [1501

In [92]:
print(result_list, )

[[0.027633275885906132, 7.106920401670655e-08]]


In [93]:
from scipy import interpolate

x_ADM = result_ADM_np[:,1]
y_ADM = result_ADM_np[:,0]

tck_ADM = interpolate.splrep(x_ADM, y_ADM)
y_spline_ADM = interpolate.splev(t_full, tck_ADM, der=0)

x_DNN = result_DNN_np[:,1]
y_DNN = result_DNN_np[:,0]

tck_DNN = interpolate.splrep(x_DNN, y_DNN)
y_spline_DNN = interpolate.splev(t_full, tck_DNN, der=0)

x_DNN_endpoint = result_DNN_np[:,1]
y_DNN_endpoint = pred_sample_endpoint

tck_DNN_endpoint = interpolate.splrep(x_DNN_endpoint, y_DNN_endpoint)
y_spline_DNN_endpoint = interpolate.splev(t_full, tck_DNN_endpoint, der=0)

x_reference = gg_opt_red[:,1]
y_reference = gg_opt_red[:,0]

tck_reference = interpolate.splrep(x_reference, y_reference)
y_spline_reference = interpolate.splev(t_full, tck_reference, der=0)

print(y_spline_ADM, y_spline_DNN, y_spline_reference, y_spline_DNN_endpoint)

[  94.95280646  530.3489517   882.5042633  1159.52651522 1369.52348148
 1520.60293604 1620.87265289 1678.44040603 1701.41396943 1697.90111709
 1676.00962298 1643.84726111 1608.58530287 1573.64900937 1541.52713914
 1514.7084507  1495.68170259 1486.08540193 1484.15705031 1487.28389794
 1492.85319499 1498.25219166 1501.41921915 1502.49693257 1502.17906805
 1501.1593617  1500.13154965 1499.64416548 1499.6649326  1500.0163719
 1500.52100426 1501.00135056 1501.31736294 1501.47871859 1501.53252598
 1501.52589354 1501.50592974 1501.51036936 1501.53945247 1501.58404548
 1501.63501481 1501.68322685 1501.72112903 1501.74749284 1501.76267079
 1501.76701537 1501.7608791  1501.74589216 1501.72879542 1501.71760742
 1501.71862687 1501.73127306 1501.75324543 1501.78224342 1501.81596648
 1501.85211404 1501.88838556 1501.92248048 1501.95209823 1501.97493826
 1501.98930104 1501.99589118 1501.99601431 1501.99097608 1501.9820821
 1501.97063802 1501.95794947 1501.94532209 1501.93406151 1501.92547337
 1501.92

In [94]:
set_dil = np.matlib.repmat(np.array(dil_fun), 16, 1)

In [100]:
# handles = ['1', '2', '3', '4', '5'] 
# labels = ['1', '2', '3', '4', '5'] 

fig1 = plt.subplots()
ax1 = plt.subplot(211)
ax1.plot(gg_ADM_np[1:,1], gg_ADM_np_endpoint[1:,0], '-.' , label=r'$\dot m(CH_4)(steady)$ ADM1')
ax1.plot(gg_ADM_np[1:,1], y_spline_DNN_endpoint[1:], '-.', label=r'$\dot m(CH_4)(steady)$ DNN')

# ax.plot(gg_np[:,1], np.reshape(gg_np[:,0], (100,1)), 'r-.')
ax1.plot(gg_np[1:,1], gg_np[1:,0], '-.' , label=r'$\dot m(CH_4)(ungeregelt)$')
# ax1.plot(gg_ADM_np[1:,1], y_spline_reference[1:], '-.', label=r'$\dot m(CH_4)_{reference}$')
ax1.plot(gg_ADM_np[1:,1], set_repd[1:], 'r--', label=r'$\dot m(CH_4)(setpoint) $')
# ax1.set_title('Trajectory')
ax1.set_ylabel(r'$\dot m(CH_4)$ [m³/d]')

handles, labels = ax1.get_legend_handles_labels()
ax1.legend(handles, labels, loc='lower right')

ax2 = plt.subplot(212)
ax2.plot(gg_opt_red[:,1], set_dil, 'o-', label=r'$dil_{optimiert} $')
# ax2.set_title('Trajectory')
ax2.set_ylabel('Dilution [1/d]')
ax2.set_xlabel('Zeit [d]')

handles, labels = ax2.get_legend_handles_labels()
ax2.legend(handles, labels, loc='best')

plt.show()
print(handles, labels)

[<matplotlib.lines.Line2D object at 0x0000021C5A1A1E88>] ['$dil_{optimiert} $']


In [101]:
# handles = ['1', '2', '3'] 
# labels = ['1', '2', '3'] 
fig2 = plt.subplots()
ax3 = plt.subplot(211)
ax3.plot(t_full[1:], y_spline_ADM[1:], '-.', label=r'$\dot m(CH_4)(mpc)$ ADM1')
ax3.plot(t_full[1:], y_spline_DNN[1:], '-.', label=r'$\dot m(CH_4)(mpc)$ DNN')
ax3.plot(gg_ADM_np[1:,1], y_spline_reference[1:], '-.', label=r'$\dot m(CH_4)(Referenz)$')
# ax3.plot(gg_ADM_np[1:,1], set_repd[1:], 'r--')
ax3.set_ylabel(r'$\dot m(CH_4)$ [m³/d]')

handles, labels = ax3.get_legend_handles_labels()
ax3.legend(handles, labels, loc='best')

ax4 = plt.subplot(212)
ax4.plot(gg_opt_red[:,1], result_mpc_np[:,0], 'o-', label=r'$dil_{mpc}(t_i) $')
# ax2.set_title('Trajectory')
ax4.set_ylabel('Dilution [1/d]')
ax4.set_xlabel('Zeit [d]')

handles, labels = ax4.get_legend_handles_labels()
ax4.legend(handles, labels, loc='best')

# ax1.legend(loc='best')

plt.show()
print(handles, labels)

[<matplotlib.lines.Line2D object at 0x0000021C5ACD1588>] ['$dil_{mpc}(t_i) $']


In [98]:
# print(dil_fun)

handles = ['1', '2', '3', '4', '5', '6', '7', '8'] 
labels = ['1', '2', '3', '4', '5', '6', '7', '8'] 

ax1 = plt.subplot(411)
ax1.plot(gg_ADM_np[1:,1], gg_ADM_np_endpoint[1:,0], '-.' , label=r'$\dot m(CH_4)_{steady}$ $_{state} $ ADM1')
ax1.plot(gg_ADM_np[1:,1], y_spline_DNN_endpoint[1:], '-.', label=r'$\dot m(CH_4)_{steady}$ $_{state} $ DNN')
ax1.plot(gg_ADM_np[1:,1], y_spline_reference[1:], '-.', label=r'$\dot m(CH_4)_{reference}$')
ax1.plot(gg_ADM_np[1:,1], set_repd[1:], 'r--', label=r'$\dot m(CH_4)_{setpoint} $')
# ax1.set_title('Trajectory')
ax1.set_ylabel(r'$\dot m(CH_4)$ [m³/d]')

ax2 = plt.subplot(412)
ax2.plot(gg_opt_red[:,1], set_dil, 'o-', label=r'$dil_{optimized} $')
# ax2.set_title('Trajectory')
ax2.set_ylabel('Dilution [1/d]')

ax3 = plt.subplot(413)
ax3.plot(t_full[1:], y_spline_ADM[1:], '-.', label=r'$\dot m(CH_4)_{mpc}$ ADM1')
ax3.plot(t_full[1:], y_spline_DNN[1:], '-.', label=r'$\dot m(CH_4)_{mpc}$ DNN')
ax3.plot(gg_ADM_np[1:,1], y_spline_reference[1:], '-.')
ax3.plot(gg_ADM_np[1:,1], set_repd[1:], 'r--')
ax3.set_ylabel(r'$\dot m(CH_4)$ [m³/d]')

ax4 = plt.subplot(414)
ax4.plot(gg_opt_red[:,1], result_mpc_np[:,0], 'o-', label=r'$dil_{mpc} $')
# ax2.set_title('Trajectory')
ax4.set_ylabel('Dilution [1/d]')
ax4.set_xlabel('time [d]')

handles[0:3], labels[0:3] = ax1.get_legend_handles_labels()
handles[4], labels[4] = ax2.get_legend_handles_labels()
handles[5:6], labels[5:6] = ax3.get_legend_handles_labels()
handles[7], labels[7] = ax4.get_legend_handles_labels()
ax4.legend(handles, labels, loc='best')

# ax1.legend(loc='best')

plt.show()
print(handles, labels)

[<matplotlib.lines.Line2D object at 0x0000021C5AE311C8>, <matplotlib.lines.Line2D object at 0x0000021C5AEF0988>, <matplotlib.lines.Line2D object at 0x0000021C5AEFFC88>, <matplotlib.lines.Line2D object at 0x0000021C5AF0C988>, [<matplotlib.lines.Line2D object at 0x0000021C5AF0C648>], <matplotlib.lines.Line2D object at 0x0000021C5AF63708>, <matplotlib.lines.Line2D object at 0x0000021C5AF71A88>, [<matplotlib.lines.Line2D object at 0x0000021C5CD638C8>], '7', '8'] ['$\\dot m(CH_4)_{steady}$ $_{state} $ ADM1', '$\\dot m(CH_4)_{steady}$ $_{state} $ DNN', '$\\dot m(CH_4)_{reference}$', '$\\dot m(CH_4)_{setpoint} $', ['$dil_{optimized} $'], '$\\dot m(CH_4)_{mpc}$ ADM1', '$\\dot m(CH_4)_{mpc}$ DNN', ['$dil_{mpc} $'], '7', '8']


D:\Anaconda\lib\site-packages\ipykernel_launcher.py:36: UserWarning: Legend does not support [<matplotlib.lines.Line2D object at 0x0000021C5AF0C648>] instances.
A proxy artist may be used instead.
See: http://matplotlib.org/users/legend_guide.html#creating-artists-specifically-for-adding-to-the-legend-aka-proxy-artists
D:\Anaconda\lib\site-packages\ipykernel_launcher.py:36: UserWarning: Legend does not support [<matplotlib.lines.Line2D object at 0x0000021C5CD638C8>] instances.
A proxy artist may be used instead.
See: http://matplotlib.org/users/legend_guide.html#creating-artists-specifically-for-adding-to-the-legend-aka-proxy-artists
D:\Anaconda\lib\site-packages\ipykernel_launcher.py:36: UserWarning: Legend does not support '7' instances.
A proxy artist may be used instead.
See: http://matplotlib.org/users/legend_guide.html#creating-artists-specifically-for-adding-to-the-legend-aka-proxy-artists
D:\Anaconda\lib\site-packages\ipykernel_launcher.py:36: UserWarning: Legend does not suppo

In [99]:
fig, ax = plt.subplots()
ax.plot(gg_np[:,1], np.reshape(gas_sobol[:,0], (100,1)), 'r-.')
ax.plot(gg_np[:,1], np.reshape(gas_minimize[:,0], (100,1)), 'b-.')
ax.plot(gg_np[:,1], np.reshape(gg_np[:,0], (100,1)), 'y-.')
ax.set_xlabel('Time [d]')
ax.set_ylabel('Concentration [m³/d]')
plt.show()

NameError: name 'gas_sobol' is not defined

In [127]:
print(result_mpc_np[3][0])

-0.09113776027679639


In [112]:
# print(gg_ADM_np)

         Current function value: 0.000000
         Iterations: 3
         Function evaluations: 62
         Gradient evaluations: 25
         Current function value: 0.000000
         Iterations: 6
         Function evaluations: 60
         Gradient evaluations: 24
         Current function value: 0.000000
         Iterations: 8
         Function evaluations: 99
         Gradient evaluations: 43
         Current function value: 0.000000
         Iterations: 4
         Function evaluations: 62
         Gradient evaluations: 25
Optimization terminated successfully.
         Current function value: 244676.268411
         Iterations: 1
         Function evaluations: 6
         Gradient evaluations: 3
Optimization terminated successfully.
         Current function value: 3626.769905
         Iterations: 1
         Function evaluations: 6
         Gradient evaluations: 3
Optimization terminated successfully.
         Current function value: 820703.594111
         Iterations: 1
         Funct

In [46]:
def ch4_ADM_DNN(Dil_test, max_f, target_ch4, sub):
    """ADM DNN Difference"""
#     print(Dil_test)
    dil = float(Dil_test)
    sub[3] = dil
#     max_f = 2570.348355
#     target_ch4 = 1250
    DNN_test = torch.tensor(sub, dtype=torch.float)
#     gg_opt_test = eng.ADM_Sim_ML_remote(0.8147, 0.0791, 0.1062, float(Dil_test))
#     gg_opt_test = eng.ADM_Sim_ML_remote(float(sub[0]), float(sub[1]), float(sub[2]), float(Dil_test))
    
    pred_sample_opt = net(DNN_test)
    pred_sample_opt = pred_sample_opt.detach().numpy()
    return (target_ch4 - (pred_sample_opt[-1] * max_f))**2


result_list = []
target_list = []

bounds = Bounds([0], [0.2])
# nonlinear_constraint = NonlinearConstraint(hess=BFGS())

start_proc = []
ende_proc = []

# start_proc = time.process_time()
start_time = time.time()
sub = [0.8147, 0.0791, 0.1062, 0]

for iv in range(20):
    x0 = np.array([iv/100])
#     res = minimize(ch4_ADM_DNN, x0, method='nelder-mead',
#                    options={'xatol': 1e-8, 'disp': True})
    res = minimize(ch4_ADM_DNN, x0, args=(max_f, target_ch4, sub), method='BFGS', options={'disp': True})
#     result_list.append(res.x)
#     res = minimize(ch4_ADM_DNN, x0, method='trust-constr',bounds=bounds, options={'disp': True})
    result_list.append([float(res.x), res.fun])

# ende_proc = time.process_time()
end_time = time.time()
print('Systemzeit: {:5.3f}s'.format(end_time - start_time))
print(result_list)

         Current function value: 0.000000
         Iterations: 3
         Function evaluations: 62
         Gradient evaluations: 25
         Current function value: 0.000000
         Iterations: 6
         Function evaluations: 60
         Gradient evaluations: 24
         Current function value: 0.000000
         Iterations: 8
         Function evaluations: 99
         Gradient evaluations: 43
         Current function value: 0.000000
         Iterations: 4
         Function evaluations: 62
         Gradient evaluations: 25
Optimization terminated successfully.
         Current function value: 244676.268411
         Iterations: 1
         Function evaluations: 6
         Gradient evaluations: 3
Optimization terminated successfully.
         Current function value: 3626.769905
         Iterations: 1
         Function evaluations: 6
         Gradient evaluations: 3
Optimization terminated successfully.
         Current function value: 820703.594111
         Iterations: 1
         Funct

In [463]:
# gg_opt_test = eng.ADM_Sim_ML_remote(0.8147, 0.0791, 0.1062, 0.1)
# print(gg_opt_test[-1][0])

0.02691873166492691


In [474]:
x1 = [0.02, 0.1, 0.15]

len(x1)
for iv in x1:
    print(np.array(iv))

0.02
0.1
0.15


In [61]:
def ch4_ADM(Dil, max_f, target_ch4, sub):
    """ADM DNN Difference"""
#     print(Dil_test)
    dil = float(Dil)

    sub[3] = dil
    
#     max_f = 2570.348355
#     target_ch4 = 1250.0
    
#     DNN_test = torch.tensor(sub, dtype=torch.float)
    
    gg_opt_test = eng.ADM_Sim_ML_remote(0.8147, 0.0791, 0.1062, float(Dil))
#     pred_sample_opt = net(DNN_test)
#     pred_sample_opt = pred_sample_opt.detach().numpy()
    
    return (target_ch4 - np.array(gg_opt_test[-1][0]))**2


result_list_ADM = []
target_list_ADM = []

# from scipy.optimize import Bounds
bounds = Bounds([0], [0.2])

# from scipy.optimize import BFGS
# from scipy.optimize import NonlinearConstraint
# nonlinear_constraint = NonlinearConstraint(hess=BFGS())

start_proc_1 = []
ende_proc_1 = []

start_time_1  = time.time()

# x1 = [0.06, 0.1, 0.15]

# len(x1)
# for iv in x1:
#     print(x1[iv])

sub = [0.8147, 0.0791, 0.1062, 0]

for iv in range(20):
    x1_1 = np.array([iv/100])
# for iv in x1:
#     x1_1 = np.array(iv)
#     print(x1)
#     res1 = minimize(ch4_ADM, x1, method='nelder-mead',
#                    options={'xatol': 1e-8, 'disp': True})
    res1 = minimize(ch4_ADM, x1_1, args=(max_f, target_ch4, sub), method='BFGS', options={'disp': True})
#     result_list.append(res.x)
#     res1 = minimize(ch4_ADM, x1_1, method='trust-constr',bounds=bounds, options={'disp': True})
    result_list_ADM.append([float(res1.x), res1.fun])

end_time_1  = time.time()
print('Systemzeit: {:5.3f}s'.format(end_time_1 - start_time_1))
print(result_list_ADM)

         Current function value: 0.000000
         Iterations: 7
         Function evaluations: 127
         Gradient evaluations: 57
         Current function value: 0.000000
         Iterations: 6
         Function evaluations: 44
         Gradient evaluations: 16
         Current function value: 0.000000
         Iterations: 7
         Function evaluations: 76
         Gradient evaluations: 32
         Current function value: 0.000000
         Iterations: 5
         Function evaluations: 79
         Gradient evaluations: 34
Optimization terminated successfully.
         Current function value: 207348.063891
         Iterations: 1
         Function evaluations: 4
         Gradient evaluations: 2
Optimization terminated successfully.
         Current function value: 207348.063891
         Iterations: 1
         Function evaluations: 4
         Gradient evaluations: 2
Optimization terminated successfully.
         Current function value: 207348.063891
         Iterations: 1
         Fu

In [53]:
# Ergebnis der Optimierung Dilution | Funktionswert
results_min = np.array(result_list)
print(results_min)

[[ 8.90532949e-04  5.37126647e+05]
 [ 2.21935234e-02  5.90133598e-08]
 [ 2.21935230e-02  5.90133598e-08]
 [-2.78746735e-01  7.14438068e+04]
 [-3.36664972e-01  2.09087309e+04]
 [-4.94324469e-01  3.59219914e+04]
 [-9.50000000e-01  1.33616706e+06]
 [ 7.00000000e-02  1.56134629e+06]
 [ 8.00000000e-02  1.56196373e+06]
 [ 9.04221631e-02  1.56202108e+06]
 [ 1.00000000e-01  1.56244839e+06]
 [ 1.55734911e-01  1.56241258e+06]
 [ 1.20000000e-01  1.56243518e+06]
 [ 1.30000000e-01  1.56239870e+06]
 [ 1.40000000e-01  1.56237102e+06]
 [ 1.90226144e-01  1.56238586e+06]
 [ 1.60000000e-01  1.56238338e+06]
 [ 1.16017733e-01  1.56245279e+06]
 [ 1.84639929e-01  1.56235848e+06]
 [ 1.90000000e-01  1.56237217e+06]]


In [58]:
np.min(abs(results_min))

# def index_finding(results_min, min_value):
#     list_1 = [sublist[1] for sublist in results_min]
#     indices = [i for i, x in enumerate(results_min) if x == min_value]
#     return indices

# print(index_finding(results_min, np.min(abs(results_min))))

result = numpy.where(results_min == numpy.amin(results_min[:,1]))
print(result)

listOfCordinates = list(zip(result[0], result[1]))
print(listOfCordinates[0][0])

dil_fun = results_min[listOfCordinates[0][0]]

print(results_min[listOfCordinates[0][0]])

# listOfCordinates[0]

(array([1, 2], dtype=int64), array([1, 1], dtype=int64))
1
[2.21935234e-02 5.90133598e-08]


In [55]:
# bestimme den maximalen Gasstrom im Gleichgewicht

def min_value(result_list):
    list_1 = [sublist[1] for sublist in result_list]
#     print(list_1)
    return max([sublist[-1] for sublist in pred_sample])

print(max_value(pred_sample))
max_sample = max_value(pred_sample)

max_gas = max_sample * max_field[-1]
print(max_gas)
# bestimme den zugehörigen Index

def index_finding(pred_sample, max_sample):
    list_1 = [sublist[-1] for sublist in pred_sample]
    indices = [i for i, x in enumerate(list_1) if x == max_sample]
    return indices

print(index_finding(pred_sample, max_sample))

NameError: name 'max_value' is not defined

In [57]:
print(res.x)

[0.19]


In [60]:
sub = [0.8147, 0.0791, 0.1062, dil_fun[0]]
DNN_test = torch.tensor(sub, dtype=torch.float)
pred_sample_val = net(DNN_test)
print(pred_sample_val[-1].detach().numpy() * 2570.348355)

1249.9997570733449


In [238]:
# bounds_1 = [(0, 0.2)]
# results_1 = dict()
# results_1['shgo'] = optimize.shgo(ch4_ADM_DNN, bounds_1, n=20, iters=150)
# results_1['shgo']

KeyboardInterrupt: 

In [65]:
bounds_1 = [(0, 0.2)]
results_2 = dict()
start_proc = time.time()
results_2['shgo_sobol'] = optimize.shgo(ch4_ADM_DNN, bounds_1, args=(max_f, target_ch4, sub), n=20, iters=60,
                                      sampling_method='sobol')
ende_proc = time.time()
print('Systemzeit: {:5.3f}s'.format(ende_proc-start_proc))

Systemzeit: 2.449s


In [487]:

results_3 = dict()
start_proc_2 = time.time()
results_3['shgo_sobol'] = optimize.shgo(ch4_ADM, bounds_1, n=20, iters=60,
                                      sampling_method='sobol')
ende_proc_2 = time.time()
print('Systemzeit: {:5.3f}s'.format(ende_proc_2-start_proc_2))

TypeError: unsupported operand type(s) for -: 'float' and 'list'

In [66]:
results_2['shgo_sobol'].fun

1.5692005099118647e-07

In [67]:
x_sobol = float(results_2['shgo_sobol'].x)

In [74]:
x_minimize = float(dil_fun[0])
print(x_minimize)

0.022193523444705664


In [69]:
sub = [0.8147, 0.0791, 0.1062, results_2['shgo_sobol'].x]
DNN_test = torch.tensor(sub, dtype=torch.float)
pred_sample_val = net(DNN_test)
print(pred_sample_val[-1].detach().numpy() * 2570.348355)

1249.9996038686443


In [71]:
gas_sobol = np.array(eng.ADM_Sim_ML_remote(carbo, protein, fat, x_sobol))
# print(gas_sobol)
gas_sobol.shape

(100, 2)

In [75]:
gas_minimize = np.array(eng.ADM_Sim_ML_remote(carbo, protein, fat, x_minimize))
gas_minimize.shape

(100, 2)

In [76]:
fig, ax = plt.subplots()
ax.plot(gg_np[:,1], np.reshape(gas_sobol[:,0], (100,1)), 'r-.')
ax.plot(gg_np[:,1], np.reshape(gas_minimize[:,0], (100,1)), 'b-.')
ax.plot(gg_np[:,1], np.reshape(gg_np[:,0], (100,1)), 'y-.')
ax.set_xlabel('Time [d]')
ax.set_ylabel('Concentration [m³/d]')
plt.show()

In [177]:
# import numpy as np
# from scipy.optimize import minimize

# def ch4_ADM_DNN(Dil_test):
#     """ADM DNN Difference"""
#     dil = float(Dil_test)
    
#     sub = [0.8147, 0.0791, 0.1062, 0]
#     sub[3] = dil
    
#     max_f = 2570.348355
    
#     DNN_test = torch.tensor(sub, dtype=torch.float)
    
#     gg_opt_test = eng.ADM_Sim_ML_remote(0.8147, 0.0791, 0.1062, float(Dil_test))
#     pred_sample = net(DNN_test)
#     pred_sample = pred_sample.detach().numpy()
    
#     return (gg_opt_test[-1][0] - (pred_sample[-1] * max_f))**2

# x0 = np.array([0.04])
# res = minimize(ch4_ADM_DNN, x0, method='nelder-mead',
#                options={'xatol': 1e-8, 'disp': True})

# # Optimization terminated successfully.
# #          Current function value: 0.000000
# #          Iterations: 339
# #          Function evaluations: 571

# print(res.x)
# # [1. 1. 1. 1. 1.]

Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 19
         Function evaluations: 38
[0.03928055]


In [176]:
## Test der einzelnen Variablentypen

# x0 = 0.04
# # sub[-1] = x0

# sub_1 = []

# sub = [0.8147, 0.0791, 0.1062]
# # sub[3] = 0.04

# sub_1 = sub.append(x0)

# print(sub)

# max_f = 2570.348355
# DNN_test = torch.tensor(sub, dtype=torch.float)

# DNN_test = torch.tensor(sub, dtype=torch.float)
# pred_sample = net(DNN_test)

# gg_opt_test = eng.ADM_Sim_ML_remote(0.8147, 0.0791, 0.1062, float(x0))

# res_DNN = pred_sample[-1].detach().numpy()

# print(res_DNN * max_field[-1])
# print(gg_opt_test[-1][0])

# zwischen = float((res_DNN * max_f))
# zwischen_1 = float(np.array(gg_opt_test[-1][0]))

# print(zwischen)
# print(zwischen_1)

# (zwischen_1 - zwischen)**2

[0.8147, 0.0791, 0.1062, 0.04]
2036.672802666155
2034.8197917846462
2036.672802689514
2034.8197917846462


3.433649413558567

In [43]:
# x_new_1 = x_test
# print(x_new_1)

In [81]:
import numpy.matlib
size = (10000, 1)

x_new_1 = np.matlib.repmat(sub, size[0], size[1])
print(x_new_1)

[[0.8147 0.0791 0.1062 0.1338]
 [0.8147 0.0791 0.1062 0.1338]
 [0.8147 0.0791 0.1062 0.1338]
 ...
 [0.8147 0.0791 0.1062 0.1338]
 [0.8147 0.0791 0.1062 0.1338]
 [0.8147 0.0791 0.1062 0.1338]]


In [82]:
dil_set = np.linspace(0,0.20,size[0])

In [83]:
# Erstelle Liste mit möglichen Dilutionen bei Substrateingang
dil_list = []
test_list = []
x_new = []
x_t = np.zeros((size[0],4))

for iii in range(size[0]):
    x_new = x_new_1[0]
    x_new[3] = dil_set[iii]
    x_t[:][iii] = x_new 

In [84]:
# # Erstelle Liste mit möglichen Dilutionen bei Substrateingang

# dil_list = []
# test_list = []
# x_new = []
# x_t = np.zeros((size[0],4))
# # print(x_t)
# for iii in range(size[0]):
#     dil_list.append((iii+1)/size[0])
# #     x_new = x_new_1[0].numpy()
#     x_new = x_new_1[0]
#     x_new[3] = (iii+1)/size[0]
#     test_list.append(x_new[:])
# #     print(x_new)
#     x_t[:][iii] = x_new 

In [85]:
print(x_t)

[[8.14700000e-01 7.91000000e-02 1.06200000e-01 0.00000000e+00]
 [8.14700000e-01 7.91000000e-02 1.06200000e-01 2.00020002e-05]
 [8.14700000e-01 7.91000000e-02 1.06200000e-01 4.00040004e-05]
 ...
 [8.14700000e-01 7.91000000e-02 1.06200000e-01 1.99959996e-01]
 [8.14700000e-01 7.91000000e-02 1.06200000e-01 1.99979998e-01]
 [8.14700000e-01 7.91000000e-02 1.06200000e-01 2.00000000e-01]]


In [86]:
# Wandle Liste von Listen in Tensorwerte um
x_test_new = torch.tensor(x_t, dtype=torch.float)

In [87]:
x_test_new[52]

tensor([0.8147, 0.0791, 0.1062, 0.0010])

In [88]:
# Nutze das Netz um Vorhersagen für die verschiedenen Dilutionen bei Substratwerten zu machen
pred_sample = net(x_test_new)
pred_sample = pred_sample.detach().numpy()
print(pred_sample)
pred_sample.shape

# schreibe die Ergebnisse der Methangasmenge in csv-file
import csv

with open("out.csv", "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerows(pred_sample)

[[ 1.4948666e-01  2.7012298e-01  1.8675372e-01 ...  2.1281400e-01
   2.1462199e-01  2.0113951e-01]
 [ 1.4937021e-01  2.6993647e-01  1.8672803e-01 ...  2.1274930e-01
   2.1456936e-01  2.0114046e-01]
 [ 1.4925356e-01  2.6974982e-01  1.8670233e-01 ...  2.1268451e-01
   2.1451662e-01  2.0114140e-01]
 ...
 [ 2.9099911e-01  4.0772557e-04 -1.3835728e-04 ... -1.5135109e-04
  -1.4905632e-04 -1.4655292e-04]
 [ 2.9099405e-01  3.9905310e-04 -1.4588237e-04 ... -1.5814602e-04
  -1.5580654e-04 -1.5328825e-04]
 [ 2.9098898e-01  3.9023161e-04 -1.5343726e-04 ... -1.6497076e-04
  -1.6260147e-04 -1.6005337e-04]]


In [89]:
# bestimme den maximalen Gasstrom im Gleichgewicht

def max_value(pred_sample):
    list_1 = [sublist[-1] for sublist in pred_sample]
#     print(list_1)
    return max([sublist[-1] for sublist in pred_sample])

print(max_value(pred_sample))
max_sample = max_value(pred_sample)

max_gas = max_sample * max_field[-1]
print(max_gas)
# bestimme den zugehörigen Index

def index_finding(pred_sample, max_sample):
    list_1 = [sublist[-1] for sublist in pred_sample]
    indices = [i for i, x in enumerate(list_1) if x == max_sample]
    return indices

print(index_finding(pred_sample, max_sample))

0.97502023
2506.1416437716566
[2626]


In [90]:
# Gegenprobe

x_point = torch.tensor(x_t[:][index_finding(pred_sample, max_sample)], dtype=torch.float)
pred_sample_point = net(x_point)
pred_sample_point = pred_sample_point.detach().numpy()
print(pred_sample_point)

print(x_t[index_finding(pred_sample, max_sample)])

dil_opt = x_t[index_finding(pred_sample, max_sample),-1]
print(dil_opt)

[[0.3631046  0.99394935 0.8814493  0.9764217  0.9767894  0.9763525
  0.9758691  0.9759009  0.97562957 0.9755889  0.97503865 0.97598356
  0.97576016 0.97582316 0.9754733  0.97502023]]
[[0.8147     0.0791     0.1062     0.05252525]]
[0.05252525]


In [91]:
dil_opt_scalar = np.ndarray.item(dil_opt)
print(dil_opt_scalar)

0.05252525252525253


In [55]:
# # Rücktransformation
# dil_opt_ADM = dil_opt_scalar * max_field_dil

# print(dil_opt_ADM)

0.010499708223207734


In [56]:
# dil_opt_ADM_1 = (dil_opt_scalar - 0.001) * max_field_dil
# dil_opt_ADM_2 = (dil_opt_scalar + 0.001) * max_field_dil

In [92]:
gg_opt = eng.ADM_Sim_ML_remote(carbo, protein, fat, dil_opt_scalar)
print(gg_opt[-1])

[2518.5529310138977,300.0]


In [94]:
gg_opt_1 = eng.ADM_Sim_ML_remote(carbo, protein, fat, (dil_opt_scalar - 0.001) )
print(gg_opt_1[-1])

[2482.0921502030724,300.0]


In [99]:
gg_opt_2 = eng.ADM_Sim_ML_remote(carbo, protein, fat,  (dil_opt_scalar + 0.001))
print(gg_opt_2[-1])

[2554.56295124515,300.0]


In [100]:
gg_np_opt = np.array(gg_opt)
gg_opt_red_1 = np.array(gg_opt[1:49:5])
gg_opt_red_2 = np.array(gg_opt[49:len(t_full):10])

gg_opt_red = np.concatenate((gg_opt_red_1 ,gg_opt_red_2), axis=0)

In [101]:
print(gg_np_opt, gg_opt_red)

[[1705.35487687    0.        ]
 [ 912.34516192    3.03030303]
 [2278.12636276    6.06060606]
 [2385.98400552    9.09090909]
 [2518.74071505   12.12121212]
 [2696.31060606   15.15151515]
 [2827.0699134    18.18181818]
 [2592.48352189   21.21212121]
 [2508.45272033   24.24242424]
 [2509.22666228   27.27272727]
 [2510.95185194   30.3030303 ]
 [2512.3332418    33.33333333]
 [2513.45587418   36.36363636]
 [2514.37543456   39.39393939]
 [2515.13021019   42.42424242]
 [2515.74977299   45.45454545]
 [2516.25813485   48.48484848]
 [2516.67507873   51.51515152]
 [2517.01692417   54.54545455]
 [2517.29710771   57.57575758]
 [2517.52667804   60.60606061]
 [2517.71470761   63.63636364]
 [2517.86864618   66.66666667]
 [2517.99461239   69.6969697 ]
 [2518.09763282   72.72727273]
 [2518.1818381    75.75757576]
 [2518.25062297   78.78787879]
 [2518.30677711   81.81818182]
 [2518.35259218   84.84848485]
 [2518.38994975   87.87878788]
 [2518.4203939    90.90909091]
 [2518.44519133   93.93939394]
 [2518.4

In [408]:
norm_gg_opt = gg_opt_red[:,0]
gg_norm_opt = []
dil_norm_opt = []

gg_norm_opt.append(norm_gg_opt[:]/max_field.values) 
print(gg_norm_opt)

dil_norm_opt.append(dil_opt_ADM/max_field_dil)
print(dil_norm_opt)

[array([0.12292828, 0.24983801, 0.22678312, 0.25135212, 0.25131813,
       0.25134225, 0.2513754 , 0.25140355, 0.25142432, 0.25143875,
       0.25144505, 0.25145772, 0.25146321, 0.25146555, 0.25146655,
       0.25146697])]
[0.0525]


In [109]:
fig, ax = plt.subplots()
ax.plot(gg_np[:,1], np.reshape(gg_np[:,0], (100,1)), 'r-.')
ax.plot(gg_np[:,1], np.reshape(gg_np_opt[:,0], (100,1)), 'b-.')
ax.plot(gg_opt_red[:,1], np.reshape(gg_opt_red[:,0], (16,1)), 'gx')
ax.set_xlabel('Time [d]')
ax.set_ylabel('Concentration [m³/d]')
plt.show()

In [9]:
%%file multi_lin_func.m

function p = multi_lin_func(a,b)
    % in-file comments can be added like this
    a = a+1;
    b = b+1;
    p = a+b;
end

Overwriting multi_lin_func.m


In [10]:
multi_lin_func(1,2)

NameError: name 'multi_lin_func' is not defined

# ab hier normaler Code

In [197]:
# Testing a sample of testdata
pred_sample = net(x_test[0])
pred_sample = pred_sample.detach().numpy()
y_real = y_test[0].detach().numpy()

print(x_test[0])
print(pred_sample)
print(y_real)


# showing deviation
print(y_real - pred_sample)


time_list = []
for ii in range(16):
    time_list.append('time_' + str(ii+1))

time_series = data_raw[time_list]
t = time_series.loc[0].to_numpy()

plt.scatter(t, y_real)
plt.scatter(t, pred_sample)
plt.show()

tensor([0.8147, 0.0791, 0.1062, 0.1338])
[ 6.4616990e-01 -4.7296286e-05  1.5363097e-05  2.0101666e-05
  2.0295382e-05  1.9475818e-05  1.8432736e-05  1.8864870e-05
  1.8239021e-05  1.7896295e-05  1.6897917e-05  1.9192696e-05
  1.8596649e-05  1.8909574e-05  1.8209219e-05  1.7240644e-05]
[6.4522964e-01 6.8613990e-05 9.6559243e-06 7.8648100e-06 7.5854050e-06
 7.5538346e-06 7.5499147e-06 7.5493431e-06 7.5492367e-06 7.5492103e-06
 7.5492048e-06 7.5491998e-06 7.5491994e-06 7.5491994e-06 7.5491994e-06
 7.5491994e-06]
[-9.40263271e-04  1.15910276e-04 -5.70717293e-06 -1.22368565e-05
 -1.27099765e-05 -1.19219831e-05 -1.08828217e-05 -1.13155274e-05
 -1.06897842e-05 -1.03470848e-05 -9.34871241e-06 -1.16434958e-05
 -1.10474502e-05 -1.13603746e-05 -1.06600201e-05 -9.69144457e-06]


In [20]:
loss = my_loss(prediction_test, y_test)
print(loss)

tensor(85.5266, grad_fn=<SumBackward0>)


In [21]:
score_model_pre = 100-(abs((prediction_test - y_test)/y_test))
score_model = torch.mean(score_model_pre)
print(score_model)

prediction_train = net(x_data)
score_model_pre = 100-(abs((prediction_train - y_data)/y_data))
score_model = torch.mean(score_model_pre)
print(score_model)

print(prediction_test.detach().numpy())
print(y_test.detach().numpy())

abw = ((prediction_test.detach().numpy() - y_test.detach().numpy())**2)
print(abw.sum())
print(abw.shape)

max_list = []
for ii in range(16):
    max_list.append(max(abw[ii,:]))
print(max_list)

tensor(-45.1772, grad_fn=<MeanBackward0>)
tensor(96.6993, grad_fn=<MeanBackward0>)
[[ 6.4616984e-01 -4.7266483e-05  1.5363097e-05 ...  1.8894672e-05
   1.8209219e-05  1.7240644e-05]
 [ 8.7011850e-01  4.1598082e-04  3.4973025e-05 ...  1.0550022e-05
   1.0460615e-05  1.0371208e-05]
 [ 3.4827200e-01  1.2359023e-04  2.6479363e-05 ...  1.7613173e-05
   1.7985702e-05  1.8715858e-05]
 ...
 [ 6.4195526e-01  1.5226007e-04  2.2202730e-05 ...  2.3275614e-05
   2.2545457e-05  2.1561980e-05]
 [ 3.3384410e-01  2.1347404e-04  2.9742718e-05 ...  1.5795231e-05
   1.6406178e-05  1.7508864e-05]
 [ 3.2752940e-01  1.5044212e-04 -1.0073185e-05 ... -2.1666288e-05
  -2.0742416e-05 -1.9431114e-05]]
[[6.45229638e-01 6.86139902e-05 9.65592426e-06 ... 7.54919938e-06
  7.54919938e-06 7.54919938e-06]
 [8.67476821e-01 2.62590096e-04 1.99666647e-05 ... 9.37213827e-06
  9.37213827e-06 9.37213827e-06]
 [3.47151935e-01 1.24998696e-05 4.75467732e-06 ... 4.95788299e-06
  4.95788299e-06 4.95788299e-06]
 ...
 [6.45257831e-0

In [164]:
import numpy as np
import numpy.matlib
# import mplcursors

time_list = []
for ii in range(16):
    time_list.append('time_' + str(ii+1))

time_series = data_raw[time_list]


t = time_series.loc[0].to_numpy()
time_series.head()

,time_1,time_2,time_3,time_4,time_5,time_6,time_7,time_8,time_9,time_10,time_11,time_12,time_13,time_14,time_15,time_16
0,3.030303,18.181818,33.333333,48.484848,63.636364,78.787879,93.939394,109.090909,124.242424,139.393939,148.484848,178.787879,209.090909,239.393939,269.69697,300
1,3.030303,18.181818,33.333333,48.484848,63.636364,78.787879,93.939394,109.090909,124.242424,139.393939,148.484848,178.787879,209.090909,239.393939,269.69697,300
2,3.030303,18.181818,33.333333,48.484848,63.636364,78.787879,93.939394,109.090909,124.242424,139.393939,148.484848,178.787879,209.090909,239.393939,269.69697,300
3,3.030303,18.181818,33.333333,48.484848,63.636364,78.787879,93.939394,109.090909,124.242424,139.393939,148.484848,178.787879,209.090909,239.393939,269.69697,300
4,3.030303,18.181818,33.333333,48.484848,63.636364,78.787879,93.939394,109.090909,124.242424,139.393939,148.484848,178.787879,209.090909,239.393939,269.69697,300


In [23]:
%matplotlib qt

In [167]:
from mpl_toolkits.mplot3d import Axes3D
y1 = y_test.numpy()
y2 = prediction_test.detach().numpy()
t = time_series.loc[0].to_numpy()
# t = np.matlib.repmat(t, len(y1), 1)
print(y1[0,3])
print(y2[0,3])

for i in range(10,20):
    fig = plt.figure()
    plt.plot(t, y1[i+10,:], 'bo--', label=r'$\dot m(CH_4)$ ADM1' , alpha=0.5)
    plt.plot(t, y2[i+10,:], marker=6, color='r', label=r'$\dot m(CH_4)$ ML', alpha=0.5)
    plt.legend(loc='best')
    plt.xlabel('time [d]')
    plt.ylabel(r'$\dot m(CH_4)$ [m³/d]')
    plt.show()

7.86481e-06
2.0116568e-05


In [69]:
# path = 'C:/Users/david.wagner/Desktop/Biomasse-Institut/Machine_Learning_Corr/checkpoint_ohneAuswasch_16dim_1.pth.tar'
# checkpoint = {'x_test': x_test, 'y_test': y_test, 'x_data': x_data, 'y_data': y_data,
#               'model': model,
#               'state_dict': model.state_dict()}

# torch.save(checkpoint, path)

In [70]:
# checkpoint_1 = torch.load(path,  map_location=lambda storage, loc: storage)

In [71]:
# x_data_1 = checkpoint['x_data'] 

In [73]:
# print(x_data_1, x_data)

tensor([[0.4294, 0.0866, 0.4840, 1.0000],
        [0.4852, 0.3456, 0.1692, 0.9925],
        [0.4716, 0.4852, 0.0433, 0.5376],
        ...,
        [0.6208, 0.3367, 0.0425, 0.4188],
        [0.3414, 0.5566, 0.1020, 0.5719],
        [0.6203, 0.2340, 0.1457, 0.9593]]) tensor([[0.4294, 0.0866, 0.4840, 1.0000],
        [0.4852, 0.3456, 0.1692, 0.9925],
        [0.4716, 0.4852, 0.0433, 0.5376],
        ...,
        [0.6208, 0.3367, 0.0425, 0.4188],
        [0.3414, 0.5566, 0.1020, 0.5719],
        [0.6203, 0.2340, 0.1457, 0.9593]])
